In [1]:

import pandas as pd
import plotly.express as px
from scipy.stats.mstats import trimmed_var
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


In [2]:
def wrangle(filepath):
    df = pd.read_csv(filepath)
    mask_1 = df['NETWORTH'] < 2e6
    mask_2 = df['TURNDOWN'] == 1
    df = df[mask_1 & mask_2]
    
    return df

In [4]:
df = wrangle(r"C:\Users\USER\Documents\Project_setup\WorldQuant\Project_6\data\scfp2019excel.zip")
print(df.shape)
df.head()

(2551, 351)


,YY1,Y1,WGT,HHSEX,AGE,AGECL,EDUC,EDCL,MARRIED,KIDS,...,NWCAT,INCCAT,ASSETCAT,NINCCAT,NINC2CAT,NWPCTLECAT,INCPCTLECAT,NINCPCTLECAT,INCQRTCAT,NINCQRTCAT
150,31,311,4730.103425,1,31,1,9,3,1,2,...,1,3,2,3,2,1,6,6,3,3
151,31,312,4741.033012,1,31,1,9,3,1,2,...,1,3,2,3,2,1,6,6,3,3
152,31,313,4733.414115,1,31,1,9,3,1,2,...,1,3,2,3,2,1,6,6,3,3
153,31,314,4732.841307,1,31,1,9,3,1,2,...,1,3,2,3,2,1,6,6,3,3
154,31,315,4740.941333,1,31,1,9,3,1,2,...,1,3,2,3,2,1,6,6,3,3


# Feature Selection
- Features would be chosen based on variance

In [5]:
# Calculate variance, get 10 largest features
top_ten_var = df.var().sort_values().tail(10)
top_ten_var

ACTBUS      1.373320e+10
BUS         1.374510e+10
PLOAN1      1.568876e+10
KGTOTAL     1.658664e+10
DEBT        2.358061e+10
NHNFIN      2.674925e+10
HOUSES      2.966011e+10
NETWORTH    5.180398e+10
NFIN        6.994137e+10
ASSET       9.809634e+10
dtype: float64

In [6]:
# Create horizontal bar chart of `top_ten_var`
fig = px.bar(
    x=top_ten_var,
    y=top_ten_var.index, 
    title = "SCF: High Variance Features"
    
)
fig.update_layout(xaxis_title='Variance', yaxis_title='Feature')
fig.show()

One thing that is noticeable throughout this project is that many of the wealth indicators are highly skewed, with a few outlier households having enormous wealth. Those outliers can affect our measure of variance. There is need to check if that's the case with one of the features from `top_five_var`.

In [7]:
# Create a boxplot of `NHNFIN`
fig = px.box(
    data_frame=df, 
    x="NHNFIN", 
    title="Distribution of Non-home, Non-Financial Assets"
)
fig.update_layout(xaxis_title="Value [$]")
fig.show()

As seen above, the dataset is massively right-skewed because of the huge outliers on the right side of the distribution. Even though we already excluded households with a high net worth with our `wrangle` function, the variance is still being distorted by some extreme outliers.

One good way to deal with this is to look at the **trimmed variance**, where we remove extreme values before calculating variance. We can do this using the `trimmed_variance` function from the `SciPy` library.

In [8]:
# Calculate trimmed variance
top_ten_trim_var = df.apply(trimmed_var).sort_values(ascending=False).head(10)
top_ten_trim_var

ASSET       1.726591e+10
NFIN        1.176106e+10
HOUSES      7.533378e+09
NETWORTH    4.655633e+09
DEBT        4.257384e+09
PLOAN1      2.651050e+09
MRTHEL      2.515424e+09
NH_MORT     2.496514e+09
HOMEEQ      1.045646e+09
INCOME      5.933635e+08
dtype: float64

In [9]:
# Create horizontal bar chart of `top_ten_trim_var`
fig = px.bar(
    x = top_ten_trim_var,
    y = top_ten_trim_var.index,
    title = "SCF: High Variance Features"
)
fig.update_layout(xaxis_title="Trimmed Variance", yaxis_title="Feature")
fig.show()

There are three things to notice in this plot. First, the variances have decreased a lot. In our previous chart, the x-axis went up to \\$80 billion; this one goes up to \\$12 billion. Second, the top 10 features have changed a bit. All the features relating to business ownership (`"...BUS"`) are gone. Finally, it can be see that there are big differences in variance from feature to feature. For example, the variance for `"WAGEINC"` is around than \\$500 million, while the variance for `"ASSET"` is nearly \\$12 billion. In other words, these features have completely different scales. This is something that needs to addressed before making clusters, one good ay is using the StandardScaler.

In [10]:
high_var_cols = top_ten_trim_var.head(5).index.to_list()
high_var_cols

['ASSET', 'NFIN', 'HOUSES', 'NETWORTH', 'DEBT']

# Split

In [12]:
X = df[high_var_cols]
print("X shape:", X.shape)
X.head()

X shape: (2551, 5)


,ASSET,NFIN,HOUSES,NETWORTH,DEBT
150,56185.0,53000.0,0.0,-7215.0,63400.0
151,55090.0,51900.0,0.0,-9310.0,64400.0
152,55200.0,52000.0,0.0,-10200.0,65400.0
153,55100.0,51900.0,0.0,-9300.0,64400.0
154,55185.0,52000.0,0.0,-8215.0,63400.0


# Modelling

In [13]:
n_clusters = range(2,13)
inertia_errors = []
silhouette_scores = []

# Add `for` loop to train model and calculate inertia, silhouette score.
for k in n_clusters:
    #make model pipeline
    model = make_pipeline(StandardScaler(), KMeans(n_clusters=k, random_state=42))
    #Train model
    model.fit(X) 
    # Cal model inertial and append to list
    inertia_errors.append(model.named_steps['kmeans'].inertia_)
    # Calculate silhouette score and append to list
    silhouette_scores.append(silhouette_score(
        X, 
        model.named_steps['kmeans'].labels_)
        )
    
print("Inertia:", inertia_errors[:3])
print()
print("Silhouette Scores:", silhouette_scores[:3])

Inertia: [6524.131658131042, 4034.8563736936485, 3303.379438482993]

Silhouette Scores: [0.7088905822791729, 0.6628617863534451, 0.6458280724081876]


In [14]:
# Create line plot of `inertia_errors` vs `n_clusters`
fig = px.line(
    x=n_clusters, 
    y=inertia_errors, 
    title= "K-Means Model: Inertia vs Number of Clusters"
)
fig.update_layout(xaxis_title="Number of Clusters", yaxis_title="Inertia")
fig.show()

In [15]:
# Create a line plot of `silhouette_scores` vs `n_clusters`
fig = px.line(
    x=n_clusters, 
    y=silhouette_scores, 
    title= "Silhouette Score vs Number of Clusters"
)
fig.update_layout(
    xaxis_title="Number of Clusters", yaxis_title="Silhouette Score")

fig.show()

Based on the above plots, using 4 clusters is reasonable.

In [16]:
final_model = make_pipeline(
    StandardScaler(), 
    KMeans(n_clusters=4, random_state=42)
)
final_model.fit(X)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kmeans', KMeans(n_clusters=4, random_state=42))])

In [17]:
labels = final_model.named_steps['kmeans'].labels_
print(labels[:5])

[0 0 0 0 0]


In [18]:
xgb = X.groupby(labels).mean()
xgb

,ASSET,NFIN,HOUSES,NETWORTH,DEBT
0,4.606971e+04,3.262453e+04,15808.776596,14729.694681,31340.018617
1,3.666664e+05,3.202373e+05,258969.135802,131358.121399,235308.302469
2,8.054979e+05,5.445849e+05,247132.812500,697602.250000,107895.625000
3,1.528714e+06,1.316484e+06,832017.543860,778612.982456,750101.403509


In [19]:
# Create side-by-side bar chart of `xgb`
fig = px.bar(
    xgb,
    barmode = 'group',
    title = "Mean Household Finances by Cluster"
)
fig.update_layout(xaxis_title = "Cluster", yaxis_title="Value [$]")
fig.show()

Since thenclusters are based partially on `NETWORTH`, which means that the households in the 0 cluster have the smallest net worth, and the households in the 2 cluster have the highest. Based on that, there are some interesting things to unpack here.

First, take a look at the `DEBT` variable. You might think that it would scale as net worth increases, but it doesn't. The lowest amount of debt is carried by the households in cluster 2, even though the value of their houses (shown in green) is roughly the same. You can't *really* tell from this data what's going on, but one possibility might be that the people in cluster 2 have enough money to pay down their debts, but not quite enough money to leverage what they have into additional debts. The people in cluster 3, by contrast, might not need to worry about carrying debt because their net worth is so high. 

Finally, it would be interesting to take a look at the relationship between `DEBT` and `HOUSES`. The value of the debt for the people in cluster 0 is higher than the value of their houses, suggesting that most of the debt being carried by those people is tied up in their mortgages — if they own a home at all. Contrast that with the other three clusters: the value of everyone else's debt is lower than the value of their homes.


In [20]:
# Instantiate transformer
pca = PCA(n_components=2, random_state=42)

# Transform `X`
X_t = pca.fit_transform(X)

# Put `X_t` into DataFrame
X_pca = pd.DataFrame(X_t, columns=['PC1','PC2'])

print("X_pca shape:", X_pca.shape)
X_pca.head()

X_pca shape: (2551, 2)


,PC1,PC2
0,-197719.042163,10944.997437
1,-199622.184630,12895.487290
2,-199626.285528,14101.193275
3,-199611.697167,12887.753571
4,-199297.207027,11562.428277


In [21]:
labelss = pd.DataFrame(labels).value_counts()
labelss

0    1880
1     486
2     128
3      57
dtype: int64

In [22]:
# Create scatter plot of `PC2` vs `PC1`
fig = px.scatter(
    data_frame = X_pca,
    x='PC1',
    y='PC2',
    color = labels.astype(str),
    title="PCA Representation of Clusters"
)
fig.update_layout(xaxis_title='PC1', yaxis_title='PC2')
fig.show()